In [ ]:
from based_plots import plotit

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# show decimal style and not scientific notation
pd.set_option("float_format",'{:,.10f}'.format)
pd.set_option('display.max_rows', 1000)

In [ ]:
results = pd.read_csv('lp_usd_seconds_per_user.csv')


In [ ]:
# look at the top 20 LPs
results.sort_values(by=['lp_usd_per_30_days'], ascending=False).head(20)

# explanations of fields
# illustrative means it's provided only for illustrative purposes, and doesn't contribute to the final calculation
# all_pool_usd_balance_for_address = maximum USD amount of liquidity provided (illustrative)
# lp_usd_seconds = total contribution measured in: USD * seconds across all points in time
# lp_usd_seconds_share = lp_usd_seconds / sum(lp_usd_seconds) for all LPs
# lp_usd_seconds_per_30_days = lp_usd_seconds/60/60/24/30 (illustrative, to scale to a common sense number)
# lp_seconds = total seconds for which the address LP'd
# lp_days = lp_seconds/60/60/24 (illustrative, total days days for which the address LP'd)
# first_datetime = illustrative, first time this address LP'd


In [ ]:
# check out distribution of results
display(results.describe(np.arange(0,1.05,0.05)))


In [ ]:
fig, axs = plt.subplots(1,1,figsize=(12,6))
plot_handle, gini_coeff, title = plotit(axs,results, 'lp_usd_seconds')
axs.set_xlabel('# of LPs')
axs.set_ylabel('share of contribution')
axs.legend(['actual distribution','perfectly equal distribution'])
plt.show()

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(12,12))
distrib = results.sort_values(by='lp_usd_seconds', ascending=False).reset_index(drop=True)
distrib["lp_usd_seconds_cum"]=distrib.lp_usd_seconds.cumsum()
distrib["lp_usd_seconds_share"]=distrib.lp_usd_seconds/sum(distrib.lp_usd_seconds)*1
distrib["lp_usd_seconds_share_cum"]=distrib.lp_usd_seconds_share.cumsum()
axs.plot(distrib.index,distrib.lp_usd_seconds_share_cum)
axs.set_title('total lp_usd_seconds')
y_lines = np.arange(0.1,1.1,0.1)
for line in y_lines:
    axs.axhline(y=line,color='red',linestyle='dotted',alpha=0.5)
x = [0,1,2,4,9,24,49,99,199]
y = []
for i in x:
    y.append(distrib["lp_usd_seconds_share_cum"][i])
    n = x.index(i)
    text = '{} users w/ {:,.1%} share'.format(i+1,y[n])
    print('{:3.0f} users w/ {:9,.1%} share'.format(i+1,y[n]))
    scale_back = 1e7
    if n>5:
        xOffset = -38
        yOffset = 0.015
    else:
        xOffset = +2
        yOffset = -0.07*1e6/scale_back
    if n>2: yOffset+= -0.05*1e6/scale_back
    plt.annotate(text,(i+xOffset,y[n]+yOffset))
plt.scatter(x, y,color='red')
plt.setp(axs,'xlim',(-2,210))
# plt.setp(axs,'ylim',(-0.1*1e6, 6*1e6))
plt.setp(axs,xticks = np.concatenate([np.arange(0,210,10)]))
plt.setp(axs,yticks = y_lines)

plt.show()